In [4]:
from huggingface_hub import notebook_login

# Authenticate to Hugging Face in a notebook environment
notebook_login()

In [5]:
from PIL import Image
import io
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi, HfFolder
from toolbox.printing import debug

df = pd.read_csv("dataset.csv")
train_df, valid_df = train_test_split(df, test_size=0.2)


def load_image(image_path):
    if image_path.lower().endswith(".png") or image_path.lower().endswith(".jpg"):
        with open(image_path, "rb") as f:
            return Image.open(io.BytesIO(f.read()))
    else:
        with open(image_path, "rb") as f:
            return io.BytesIO(f.read())


def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()


def transform(row):
    row["tex_code"] = read_file(row["tex_code"])
    for key in ["output"]:
        if row[key] is not None and len(row[key]) > 0:
            row[key] = load_image(row[key])
    return row


train_dataset = Dataset.from_pandas(train_df).map(transform)
valid_dataset = Dataset.from_pandas(valid_df).map(transform)

dataset_dict = DatasetDict({"train": train_dataset, "validation": valid_dataset})
# Then save or upload as before

dataset_dict.push_to_hub("JosselinSom/Latex-VLM")

Map:   0%|          | 0/1323 [00:00<?, ? examples/s]

ArrowInvalid: Could not convert <_io.BytesIO object at 0x7fdeb2230e00> with type _io.BytesIO: did not recognize Python value type when inferring an Arrow data type